In [1]:
___Author___='LumberJack Jyss'

In [2]:
print('Global Optimized LumberJack Equities Method\n°OoO_FOREX_Oo0°\nLumberJack Jyss 5779(c)')

Global Optimized LumberJack Equities Method
°OoO_FOREX_Oo0°
LumberJack Jyss 5779(c)


In [3]:
print('Importation des librairies...')
import talib
import numpy as np
import pandas as pd
import warnings
from multiprocessing import cpu_count
from statsmodels.tsa.arima_model import ARIMA
from statsmodels.tsa.statespace.sarimax import SARIMAX
from sklearn.metrics import mean_squared_error
from joblib import Parallel
from joblib import delayed
import matplotlib
import matplotlib.pyplot as plt
matplotlib.use('TkAgg')
import pylab
import seaborn as sns
%matplotlib inline
print('Librairies importées')

Importation des librairies...
Librairies importées


In [4]:
# SCRAPING DES DONNES BRUTES
gbpjpy = pd.read_csv('gbpjpy_15mn_1Y.csv')
gbpjpy = gbpjpy.set_index('Date')
gbpjpy = gbpjpy.iloc[:,1:-1]
gbpusd = pd.read_csv('gbpusd_15mn_1Y.csv')
gbpusd = gbpusd.set_index('Date')
gbpusd = gbpusd.iloc[:-2,1:-1]
eurusd = pd.read_csv('eurusd_15mn_1Y.csv')
eurusd = eurusd.set_index('Date')
eurusd = eurusd.iloc[:-2,1:-1]
len_gbpjpy = len(gbpjpy)
len_gbpusd = len(gbpusd)
len_eurusd = len(eurusd)
gbpjpy_norm = []
gbpusd_norm = []
eurusd_norm = []

In [5]:
gbpjpy.head()

,Open,High,Low,Close
Date,,,,
20180506 23:15:00,147.6950,147.6995,147.6840,147.6910
20180506 23:30:00,147.6910,147.7155,147.6435,147.6855
20180506 23:45:00,147.6855,147.6885,147.6455,147.6705
20180507 00:00:00,147.6705,147.6755,147.6060,147.6275
20180507 00:15:00,147.6275,147.6505,147.6015,147.6370


In [6]:
gbpusd.head()

,Open,High,Low,Close
Date,,,,
20180506 23:15:00,1.353625,1.353650,1.353490,1.353550
20180506 23:30:00,1.353550,1.353795,1.353245,1.353505
20180506 23:45:00,1.353505,1.353610,1.353245,1.353560
20180507 00:00:00,1.353560,1.353645,1.353235,1.353605
20180507 00:15:00,1.353605,1.353870,1.353515,1.353770


In [7]:
eurusd.head()

,Open,High,Low,Close
Date,,,,
20180506 23:15:00,1.195975,1.196370,1.195975,1.196155
20180506 23:30:00,1.196155,1.196205,1.196020,1.196060
20180506 23:45:00,1.196060,1.196325,1.195575,1.195805
20180507 00:00:00,1.195805,1.196170,1.195780,1.195980
20180507 00:15:00,1.195980,1.196360,1.195960,1.196255


In [8]:
close_gbpjpy = pd.DataFrame(index=gbpjpy.index)
close_gbpjpy['Close'] = gbpjpy['Close']
close_gbpjpy.head()

,Close
Date,
20180506 23:15:00,147.6910
20180506 23:30:00,147.6855
20180506 23:45:00,147.6705
20180507 00:00:00,147.6275
20180507 00:15:00,147.6370


In [ ]:
plt.figure(figsize=(16, 6), dpi=100)
plt.plot(close_gbpjpy.index, close_gbpjpy.values, label='Close du gbpjpy')
#plt.vlines(datetime.date(20180506), 0, 270, linestyles='--', colors='gray', label='Train/Test data cut-off')
plt.xlabel('Date')
plt.ylabel('Devise')
plt.title('Figure 2: GBP JPY')
plt.legend()
plt.show()

In [ ]:
# evaluate an ARIMA model for a given order (p,d,q)
def evaluate_arima_model(X, arima_order):
	# prepare training dataset
	train_size = int(len(X) * 0.66)
	train, test = X[0:train_size], X[train_size:]
	history = [x for x in train]
	# make predictions
	predictions = list()
	for t in range(len(test)):
		model = ARIMA(history, order=arima_order)
		model_fit = model.fit(disp=0)
		yhat = model_fit.forecast()[0]
		predictions.append(yhat)
		history.append(test[t])
	# calculate out of sample error
	error = mean_squared_error(test, predictions)
	return error

# evaluate combinations of p, d and q values for an ARIMA model
def evaluate_models(dataset, p_values, d_values, q_values):
	dataset = dataset.astype('float32')
	best_score, best_cfg = float("inf"), None
	for p in p_values:
		for d in d_values:
			for q in q_values:
				order = (p,d,q)
				try:
					mse = evaluate_arima_model(dataset, order)
					if mse < best_score:
						best_score, best_cfg = mse, order
					print('ARIMA%s MSE=%.3f' % (order,mse))
				except:
					continue
	print('Best ARIMA%s MSE=%.3f' % (best_cfg, best_score))

# load dataset
def parser(x):
	return datetime.strptime('190'+x, '%Y-%m')
series = close_gbpjpy['Close']
# evaluate parameters
p_values = [0, 1, 2, 4, 6, 8, 10]
d_values = range(0, 3)
q_values = range(0, 3)
warnings.filterwarnings("ignore")
evaluate_models(series.values, p_values, d_values, q_values)

ARIMA(0, 0, 0) MSE=9.599
ARIMA(0, 0, 1) MSE=2.500
ARIMA(0, 1, 0) MSE=0.010
ARIMA(0, 1, 1) MSE=0.010
ARIMA(0, 1, 2) MSE=0.010
ARIMA(0, 2, 0) MSE=0.021
ARIMA(0, 2, 1) MSE=0.010
ARIMA(1, 0, 0) MSE=0.010
ARIMA(1, 1, 0) MSE=0.010
ARIMA(1, 2, 0) MSE=0.016
ARIMA(2, 1, 0) MSE=0.010
ARIMA(2, 2, 0) MSE=0.014
ARIMA(4, 1, 0) MSE=0.010
ARIMA(4, 1, 1) MSE=0.010
ARIMA(4, 1, 2) MSE=0.010
ARIMA(4, 2, 0) MSE=0.012


In [25]:
series = close_gbpjpy['Close']
model = ARIMA(series, order=(0, 1, 2))
model_fit = model.fit(maxiter=200, method='mle',disp=0)
print(model_fit.summary())

/Users/YTsBaCh/Applications/anaconda3/envs/LumberJack/lib/python3.6/site-packages/statsmodels/tsa/base/tsa_model.py:225: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  ' ignored when e.g. forecasting.', ValueWarning)
/Users/YTsBaCh/Applications/anaconda3/envs/LumberJack/lib/python3.6/site-packages/statsmodels/tsa/base/tsa_model.py:225: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  ' ignored when e.g. forecasting.', ValueWarning)


                             ARIMA Model Results                              
Dep. Variable:                D.Close   No. Observations:                24480
Model:                 ARIMA(0, 1, 2)   Log Likelihood               23267.203
Method:                           mle   S.D. of innovations              0.094
Date:                Sun, 12 May 2019   AIC                         -46526.406
Time:                        21:38:07   BIC                         -46493.983
Sample:                             1   HQIC                        -46515.901
                                                                              
                    coef    std err          z      P>|z|      [0.025      0.975]
---------------------------------------------------------------------------------
const         -8.036e-05      0.001     -0.140      0.889      -0.001       0.001
ma.L1.D.Close    -0.0060      0.006     -0.945      0.345      -0.019       0.006
ma.L2.D.Close    -0.0334      0.006     

In [26]:
X = series.values
size = int(len(X) * 0.66)
train, test = X[0:size], X[size:len(X)]
history = [x for x in train]
predictions = list()
minlist = []
maxlist = []
diff = []
doubledelta=[]
for t in range(len(test)):
    model = ARIMA(history, order=(0,1,2))
    model_fit = model.fit(maxiter=200, method='mle',disp=0)
    output = model_fit.forecast()
    yhat = output[0]
    #minhat = tuple(tuple(output)[2])[0][0]
    #maxhat = tuple(tuple(output)[2])[0][1]
    predictions.append(yhat)
    #minlist.append(minhat)
    #maxlist.append(maxhat)
    obs = test[t]
    history.append(obs)
    diff.append((obs-yhat))
    #doubledelta.append((maxhat-minhat)-(obs-yhat))

/Users/YTsBaCh/Applications/anaconda3/envs/LumberJack/lib/python3.6/site-packages/statsmodels/base/model.py:508: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  "Check mle_retvals", ConvergenceWarning)
/Users/YTsBaCh/Applications/anaconda3/envs/LumberJack/lib/python3.6/site-packages/statsmodels/base/model.py:508: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  "Check mle_retvals", ConvergenceWarning)
/Users/YTsBaCh/Applications/anaconda3/envs/LumberJack/lib/python3.6/site-packages/statsmodels/base/model.py:508: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  "Check mle_retvals", ConvergenceWarning)
/Users/YTsBaCh/Applications/anaconda3/envs/LumberJack/lib/python3.6/site-packages/statsmodels/base/model.py:508: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  "Check mle_retvals", ConvergenceWarning)
/Users/YTsBaCh/A

In [27]:
error = mean_squared_error(test, predictions)
print('Test Mean Square Error : %.3f ' % error)

Test Mean Square Error : 0.010 


In [13]:
# one-step sarima forecast
def sarima_forecast(history, config):
	order, sorder, trend = config
	# define model
	model = SARIMAX(history, order=order, seasonal_order=sorder, trend=trend, enforce_stationarity=False, enforce_invertibility=False)
	# fit model
	model_fit = model.fit(disp=False)
	# make one step forecast
	yhat = model_fit.predict(len(history), len(history))
	return yhat[0]

# root mean squared error or rmse
def measure_rmse(actual, predicted):
	return sqrt(mean_squared_error(actual, predicted))

# split a univariate dataset into train/test sets
def train_test_split(data, n_test):
	return data[:-n_test], data[-n_test:]

# walk-forward validation for univariate data
def walk_forward_validation(data, n_test, cfg):
	predictions = list()
	# split dataset
	train, test = train_test_split(data, n_test)
	# seed history with training dataset
	history = [x for x in train]
	# step over each time-step in the test set
	for i in range(len(test)):
		# fit model and make forecast for history
		yhat = sarima_forecast(history, cfg)
		# store forecast in list of predictions
		predictions.append(yhat)
		# add actual observation to history for the next loop
		history.append(test[i])
	# estimate prediction error
	error = measure_rmse(test, predictions)
	return error

# score a model, return None on failure
def score_model(data, n_test, cfg, debug=False):
	result = None
	# convert config to a key
	key = str(cfg)
	# show all warnings and fail on exception if debugging
	if debug:
		result = walk_forward_validation(data, n_test, cfg)
	else:
		# one failure during model validation suggests an unstable config
		try:
			# never show warnings when grid searching, too noisy
			with catch_warnings():
				filterwarnings("ignore")
				result = walk_forward_validation(data, n_test, cfg)
		except:
			error = None
	# check for an interesting result
	if result is not None:
		print(' > Model[%s] %.3f' % (key, result))
	return (key, result)

# grid search configs
def grid_search(data, cfg_list, n_test, parallel=True):
	scores = None
	if parallel:
		# execute configs in parallel
		executor = Parallel(n_jobs=cpu_count(), backend='multiprocessing')
		tasks = (delayed(score_model)(data, n_test, cfg) for cfg in cfg_list)
		scores = executor(tasks)
	else:
		scores = [score_model(data, n_test, cfg) for cfg in cfg_list]
	# remove empty results
	scores = [r for r in scores if r[1] != None]
	# sort configs by error, asc
	scores.sort(key=lambda tup: tup[1])
	return scores

# create a set of sarima configs to try
def sarima_configs(seasonal=[0]):
	models = list()
	# define config lists
	p_params = [0, 1, 2]
	d_params = [0, 1]
	q_params = [0, 1, 2]
	t_params = ['n','c','t','ct']
	P_params = [0, 1, 2]
	D_params = [0, 1]
	Q_params = [0, 1, 2]
	m_params = seasonal
	# create config instances
	for p in p_params:
		for d in d_params:
			for q in q_params:
				for t in t_params:
					for P in P_params:
						for D in D_params:
							for Q in Q_params:
								for m in m_params:
									cfg = [(p,d,q), (P,D,Q,m), t]
									models.append(cfg)
	return models

if __name__ == '__main__':
	# define dataset
	data = close_gbpjpy['Close'].values
	print(data)
	# data split
	n_test = 4
	# model configs
	cfg_list = sarima_configs()
	# grid search
	scores = grid_search(data, cfg_list, n_test)
	print('done')
	# list top 3 configs
	for cfg, error in scores[:3]:
		print(cfg, error)

[147.691  147.6855 147.6705 ... 145.5625 145.76   145.7285]
done


[[(0, 0, 0), (0, 0, 0, 0), 'n'],
 [(0, 0, 0), (0, 0, 1, 0), 'n'],
 [(0, 0, 0), (0, 0, 2, 0), 'n'],
 [(0, 0, 0), (0, 1, 0, 0), 'n'],
 [(0, 0, 0), (0, 1, 1, 0), 'n'],
 [(0, 0, 0), (0, 1, 2, 0), 'n'],
 [(0, 0, 0), (1, 0, 0, 0), 'n'],
 [(0, 0, 0), (1, 0, 1, 0), 'n'],
 [(0, 0, 0), (1, 0, 2, 0), 'n'],
 [(0, 0, 0), (1, 1, 0, 0), 'n'],
 [(0, 0, 0), (1, 1, 1, 0), 'n'],
 [(0, 0, 0), (1, 1, 2, 0), 'n'],
 [(0, 0, 0), (2, 0, 0, 0), 'n'],
 [(0, 0, 0), (2, 0, 1, 0), 'n'],
 [(0, 0, 0), (2, 0, 2, 0), 'n'],
 [(0, 0, 0), (2, 1, 0, 0), 'n'],
 [(0, 0, 0), (2, 1, 1, 0), 'n'],
 [(0, 0, 0), (2, 1, 2, 0), 'n'],
 [(0, 0, 0), (0, 0, 0, 0), 'c'],
 [(0, 0, 0), (0, 0, 1, 0), 'c'],
 [(0, 0, 0), (0, 0, 2, 0), 'c'],
 [(0, 0, 0), (0, 1, 0, 0), 'c'],
 [(0, 0, 0), (0, 1, 1, 0), 'c'],
 [(0, 0, 0), (0, 1, 2, 0), 'c'],
 [(0, 0, 0), (1, 0, 0, 0), 'c'],
 [(0, 0, 0), (1, 0, 1, 0), 'c'],
 [(0, 0, 0), (1, 0, 2, 0), 'c'],
 [(0, 0, 0), (1, 1, 0, 0), 'c'],
 [(0, 0, 0), (1, 1, 1, 0), 'c'],
 [(0, 0, 0), (1, 1, 2, 0), 'c'],
 [(0, 0, 0

In [ ]:
gbpjpy.shape,gbpusd.shape,eurusd.shape

In [ ]:
gbpjpy_close = []
gbpjpy_oc = []
gbpjpy_hl = []
gbpjpy_Doc = []
gbpjpy_Dhl = []

gbpusd_close = []
gbpusd_oc = []
gbpusd_hl = []
gbpusd_Doc = []
gbpusd_Dhl = []

eurusd_close = []
eurusd_oc = []
eurusd_hl = []
eurusd_Doc = []
eurusd_Dhl = []

In [ ]:
rsi_gbpjpy = talib.RSI(gbpjpy['Close'], timeperiod=2)
rsi_gbpusd = talib.RSI(gbpusd['Close'], timeperiod=2)
rsi_eurusd = talib.RSI(eurusd['Close'], timeperiod=2)
len(rsi_gbpjpy),len(rsi_gbpusd),len(rsi_eurusd)

In [ ]:
for i in range (0,len_eurusd):
    gbpjpy_oc.append(float(gbpjpy.iloc[i,0]) - float(gbpjpy.iloc[i,3]))
    gbpjpy_hl.append(float(gbpjpy.iloc[i,1]) - float(gbpjpy.iloc[i,2]))
    
    gbpusd_oc.append(float(gbpusd.iloc[i,0]) - float(gbpusd.iloc[i,3]))
    gbpusd_hl.append(float(gbpusd.iloc[i,1]) - float(gbpusd.iloc[i,2]))
    
    eurusd_oc.append(float(eurusd.iloc[i,0]) - float(eurusd.iloc[i,3]))
    eurusd_hl.append(float(eurusd.iloc[i,1]) - float(eurusd.iloc[i,2]))

In [ ]:
len(gbpjpy_oc),len(gbpjpy_hl),len(gbpjpy),len(gbpusd_oc),len(gbpusd_hl),len(gbpusd),len(eurusd_oc),len(eurusd_hl),len(eurusd)

In [ ]:
gbpjpy['oc_gbpjpy'] = gbpjpy_oc
gbpjpy['hl_gbpjpy'] = gbpjpy_hl
gbpjpy.head()

In [ ]:
gbpusd['oc_gbpusd'] = gbpusd_oc
gbpusd['hl_gbpusd'] = gbpusd_hl
gbpusd.head()

In [ ]:
eurusd['oc_gbpusd'] = eurusd_oc
eurusd['hl_gbpusd'] = eurusd_hl
eurusd.head()

In [ ]:
for i in range (1,len_eurusd):
    b = i - 1
    gbpjpy_close.append((gbpjpy.iloc[i,3] - gbpjpy.iloc[b,3]) / (gbpjpy.iloc[b,3]+0.0001))
    gbpjpy_Doc.append((gbpjpy.iloc[i,4] - gbpjpy.iloc[b,4]) / (gbpjpy.iloc[b,4]+0.0001))
    gbpjpy_Dhl.append((gbpjpy.iloc[i,5] - gbpjpy.iloc[b,5]) / (gbpjpy.iloc[b,5]+0.0001))
    
    gbpusd_close.append((gbpusd.iloc[i,3] - gbpusd.iloc[b,3]) / (gbpusd.iloc[b,3]+0.0001))
    gbpusd_Doc.append((gbpusd.iloc[i,4] - gbpusd.iloc[b,4]) / (gbpusd.iloc[b,4]+0.0001))
    gbpusd_Dhl.append((gbpusd.iloc[i,5] - gbpusd.iloc[b,5]) / (gbpusd.iloc[b,5]+0.0001))
    
    eurusd_close.append((eurusd.iloc[i,3] - eurusd.iloc[b,3]) / (eurusd.iloc[b,3]+0.0001))
    eurusd_Doc.append((eurusd.iloc[i,4] - eurusd.iloc[b,4]) / (eurusd.iloc[b,4]+0.0001))
    eurusd_Dhl.append((eurusd.iloc[i,5] - eurusd.iloc[b,5]) / (eurusd.iloc[b,5]+0.0001))

In [ ]:
gbpjpy = gbpjpy.iloc[1:,:]
gbpusd = gbpusd.iloc[1:,:]
eurusd = eurusd.iloc[1:,:]

In [ ]:
len(gbpjpy),len(gbpusd),len(eurusd),len(gbpjpy_close),len(gbpusd_close),len(eurusd_close),len(gbpjpy_Doc)

In [ ]:
gbpjpy['gbpjpy_close'] = gbpjpy_close
gbpjpy['gbpjpy_Doc'] = gbpjpy_Doc
gbpjpy['gbpjpy_Dhl'] = gbpjpy_Dhl
gbpjpy['rsi_gbpjpy'] = rsi_gbpjpy

gbpusd['gbpusd_close'] = gbpusd_close
gbpusd['gbpusd_Doc'] = gbpusd_Doc
gbpusd['gbpusd_Dhl'] = gbpusd_Dhl
gbpusd['rsi_gbpusd'] = rsi_gbpusd

eurusd['eurusd_close'] = eurusd_close
eurusd['eurusd_Doc'] = eurusd_Doc
eurusd['eurusd_Dhl'] = eurusd_Dhl
eurusd['rsi_eurusd'] = rsi_eurusd

df = pd.concat((gbpjpy.iloc[:,4:],gbpusd.iloc[:,4:],eurusd.iloc[:,4:]),axis=1)
df = df.dropna()
df.shape

In [ ]:
df = gbpjpy.iloc[:,4:]
df = df.dropna()
df.shape

In [ ]:
df.head()

In [ ]:
df.tail()

In [ ]:
plt.figure(figsize=(16,6))
sns.heatmap(df.corr())
plt.show()

In [ ]:
df.head()

In [ ]:
df['GBPJPY']=gbpjpy.iloc[:,3]
#df['GBPUSD']=gbpusd.iloc[:,3]
#df['EURUSD']=eurusd.iloc[:,3]

In [ ]:
df.info()

In [ ]:
df.head()

plt.figure(figsize=(16,6))
plt.plot(df['GBPJPY'],label='GBPJPY')
plt.plot(df['rsi_gbpjpy'],label = 'RSI GBPJPY (14)')
plt.legend()
plt.show()

for row in range(1,len_df):
    if df.iloc[row-1,5] < rsi_low_long and df.iloc[row,5] > rsi_low_long :
        short_list_gbpjpy.append(0)
        long_list_gbpjpy.append(1)
    elif df.iloc[row-1,5] > rsi_high_short and df.iloc[row,5] < rsi_high_short :
        short_list_gbpjpy.append(1)
        long_list_gbpjpy.append(0)
    else:
        short_list_gbpjpy.append(0)
        long_list_gbpjpy.append(0)

# TARGET = GBPJPY
short_list_gbpjpy = []
long_list_gbpjpy = []
rsi_low_long = 5
rsi_target_long = 80
rsi_high_short = 95
rsi_target_short = 20
delta_pip = 0.05
len_df = len(df)-4

In [ ]:
short_list_gbpjpy = []
long_list_gbpjpy = []
delta_pip = 0.25
len_df = len(df)-4

for row in range(0,len_df):
    if (df.iloc[row+1,6] - df.iloc[row,6]) > delta_pip or (df.iloc[row+2,6] - df.iloc[row,6]) > delta_pip or\
    (df.iloc[row+3,6] - df.iloc[row,6]) > delta_pip:
        short_list_gbpjpy.append(0)
        long_list_gbpjpy.append(1)
        
    elif (df.iloc[row+1,6] - df.iloc[row,6]) < -delta_pip or (df.iloc[row+2,6] - df.iloc[row,6]) < -delta_pip or\
    (df.iloc[row+3,6] - df.iloc[row,6]) < -delta_pip:
        short_list_gbpjpy.append(1)
        long_list_gbpjpy.append(0)
   
    else:
        long_list_gbpjpy.append(0)
        short_list_gbpjpy.append(0)

In [ ]:
len_df, len(short_list_gbpjpy),len(long_list_gbpjpy),sum(short_list_gbpjpy),sum(long_list_gbpjpy)

long_list_gbpjpy = []
flag_long = 0
dube_long = 0
for row in range(1,len_df):
    if df.iloc[row-1,5] < rsi_low_long and df.iloc[row,5] > rsi_low_long :
        if (df.iloc[row+1,5] > rsi_target_long) and ((df.iloc[row+1,18] - df.iloc[row,18]) > delta_pip) and flag_long == 0:
            short_list_gbpjpy.append(0)
            long_list_gbpjpy.append(1)
            flag_long = 1
            dube_long = 1
        elif df.iloc[row+2,5] > rsi_target_long and (df.iloc[row+2,18] - df.iloc[row,18]) > delta_pip and flag_long == 0:
            short_list_gbpjpy.append(0)
            long_list_gbpjpy.append(1)
            flag_long = 1
            dube_long = 1
        elif df.iloc[row+3,5] > rsi_target_long and (df.iloc[row+3,18] - df.iloc[row,18]) > delta_pip and flag_long == 0:
            short_list_gbpjpy.append(0)
            long_list_gbpjpy.append(1)
            flag_long = 1
            dube_long = 1
        elif df.iloc[row+3,5] > rsi_target_long and (df.iloc[row+4,18] - df.iloc[row,18]) > delta_pip and flag_long == 0:
            short_list_gbpjpy.append(0)
            long_list_gbpjpy.append(1)
            flag_long = 1
            dube_long = 1
        elif df.iloc[row+3,5] > rsi_target_long and (df.iloc[row+5,18] - df.iloc[row,18]) > delta_pip and flag_long == 0:
            short_list_gbpjpy.append(0)
            long_list_gbpjpy.append(1)
            flag_long = 1
            dube_long = 1
        elif df.iloc[row+3,5] > rsi_target_long and (df.iloc[row+6,18] - df.iloc[row,18]) > delta_pip and flag_long == 0:
            short_list_gbpjpy.append(0)
            long_list_gbpjpy.append(1)
            flag_long = 1
            dube_long = 1
        flag_long = 0

if dube != 1:
    long_list_gbpjpy.append(0)
    short_list_gbpjpy.append(0)
    dube_short = 0

short_list_gbpjpy = []
flag_short = 0
dube_short = 0
for row in range(1,len_df):      
    if df.iloc[row-1,5] > rsi_high_short and df.iloc[row,5] < rsi_high_short :
        if df.iloc[row+1,5] < rsi_target_short and (df.iloc[row+1,18] - df.iloc[row,18]) < -delta_pip and flag_short == 0:
            short_list_gbpjpy.append(1)
            long_list_gbpjpy.append(0)
            flag_short = 1
            dube_short = 1
        elif df.iloc[row+2,5] < rsi_target_short and (df.iloc[row+2,18] - df.iloc[row,18]) < -delta_pip and flag_short == 0:
            short_list_gbpjpy.append(1)
            long_list_gbpjpy.append(0)
            flag_short = 1
            dube_short = 1
        elif df.iloc[row+3,5] < rsi_target_short and (df.iloc[row+3,18] - df.iloc[row,18]) < -delta_pip and flag_short == 0:
            short_list_gbpjpy.append(1)
            long_list_gbpjpy.append(0)
            flag_short = 1
            dube_short = 1
        elif df.iloc[row+3,5] < rsi_target_short and (df.iloc[row+4,18] - df.iloc[row,18]) < -delta_pip and flag_short == 0:
            short_list_gbpjpy.append(1)
            long_list_gbpjpy.append(0)
            flag_short = 1
            dube_short = 1
        elif df.iloc[row+3,5] < rsi_target_short and (df.iloc[row+5,18] - df.iloc[row,18]) < -delta_pip and flag_short == 0:
            short_list_gbpjpy.append(1)
            long_list_gbpjpy.append(0)
            flag_short = 1
            dube_short = 1
        elif df.iloc[row+3,5] < rsi_target_short and (df.iloc[row+6,18] - df.iloc[row,18]) < -delta_pip and flag_short == 0:
            short_list_gbpjpy.append(1)
            long_list_gbpjpy.append(0)
            flag_short = 1
            dube_short = 1
        flag_short = 0
    
    if dube != 1:
        long_list_gbpjpy.append(0)
        short_list_gbpjpy.append(0)
        dube_short = 0

In [ ]:
df = df.iloc[4:,:]

In [ ]:
len(df),len_df, len(short_list_gbpjpy),len(long_list_gbpjpy),sum(short_list_gbpjpy),sum(long_list_gbpjpy)

In [ ]:
df['short_gbpjpy'] = short_list_gbpjpy
df['long_gbpjpy'] = long_list_gbpjpy

In [ ]:
df.head()
        

In [ ]:
df.to_csv('gbpjpy.csv')